# Øvelse 1 - Linear Algebra

I dagens øvelse kommer vi til, at lave en gennemgang af nogle af de koncepter og metoder i har lært indtil nu.
Dermed bliver det et recap af linear algebra delen af Anvendt Matematik.

# 0 - Import

In [13]:
import numpy as np
from sympy import *
from scipy import linalg
from sympy.solvers.solveset import linsolve

# 1 - Lønudgifter

Sarah og James er ansat som konsulenter i virksomheden ABC.
De har følgende timepriser i DKK:

$$\begin{bmatrix} 250 & 300 \\ 500 & 450 \\ 700 & 800 \end{bmatrix}$$.

Her angiver rækkerne i matricen følgende værdier:
* Række 1: Timepris mellem 08:00 og 17:00 i hverdagene.
* Række 2: Timepris mellem 17:00 og 08:00 i hverdagene.
* Række 3: Timepris i weekenden. 

En gennemsnitlig arbejdsuge for de Sarah og James er givet ved følgende matrice:

$$\begin{bmatrix} 27 & 10 & 5 \\ 30 & 4 & 8 \end{bmatrix}$$.

Hvor rækkerne betegner følgende:
* Række 1: Sarahs arbejdstid. 
* Række 2: James' arbejdstid. 

Og kolonnerne:
* Kolonne 1: Arbejdstid mellem 08:00 og 17:00 i hverdagene.
* Kolonne 2: Arbejdstid mellem 17:00 og 08:00 i hverdagene. 
* Kolonne 3: Arbejdstid i weekenden.

Du vil som omkostningsansvarlig for Sarah og James nu gerne vide følgende:
* De samlede gennemsnitlige omkostninger om ugen til Sarah og James.
* De samlede gennemsnitlige omkostninger om måneden til Sarah og James (antag at en måned er 52 uger / 12 måneder = 4.33 uger pr. måned). 
* De gennemsnitlige omkostninger om ugen til hhv. Sarah og James.
* De gennemsnitlige omkostninger om måneden til hhv. Sarah og James.

In [79]:
# 1.1 - Definer de nødvendige matricer
timepris = np.array([
    [250, 300],
    [500, 450],
    [700, 800]
], dtype=np.float32)

arbejdstid = np.array([
    [27, 10, 5],
    [30, 4, 8]
], dtype=np.float32)

In [56]:
# 1.2 - Beregn de gennemsnitlige omkostninger til Sarah om ugen
sarah_uge = timepris[:, 0] @ arbejdstid[0]
sarah_uge

15250.0

In [57]:
# 1.3 - Beregn de gennemsnitlige omkostninger til James om ugen 
james_uge = timepris[:, 1] @ arbejdstid[1]
james_uge

17200.0

In [123]:
# 1.4 - Beregn de samlede omkostninger pr. uge.
saml_omkostninger = sum((arbejdstid @ timepris).diagonal())
saml_omkostninger

32450.0

In [125]:
(arbejdstid @ timepris).diagonal()

array([15250., 17200.], dtype=float32)

In [126]:
# 1.5 - Konverter matricen arbejdstid fra gns. arbejdstid pr. uge til gns. arbejdstid pr. måned:
arbejdstid_måned = (52 / 12) * arbejdstid
arbejdstid_måned

array([[117.00001 ,  43.333336,  21.666668],
       [130.      ,  17.333334,  34.666668]], dtype=float32)

In [60]:
# 1.7 - Beregn de gennemsnitlige omkostninger til Sarah om måneden
sarah_måned = timepris[:, 0] @ arbejdstid_måned[0]
sarah_måned

66083.34

In [39]:
# 1.8 - Beregn de gennemsnitlige omkostninger til James om måneden
james_måned = timepris[:, 1] @ arbejdstid_måned[1]
james_måned

74533.336

In [84]:
# 1.9 - Beregn de samlede omkostninger pr. måned
saml_måned = sum((arbejdstid_måned @ timepris).diagonal())
saml_måned

140616.671875

James har et ønske om, at gå 25 % ned i timer. For at kompensere for dette, ser du som leder for de to konsulenter, at Sarah er nødt til at gå 35 % op i timer pr. måned, hvilket hun har indvilliget i.
Hvordan vil dette påvirke de samlede omkostninger pr. måned?
Hvordan vil dette påvirke de samlede omkostninger til hhv. Sarah og James pr. uge?

In [85]:
# 1.10 - Justér den månedlige arbejdstid, så James har 25 % færre timer og Sarah har 35 % flere timer
arbejdstid_måned[0] = arbejdstid_måned[0] * 1.35
arbejdstid_måned[1] = arbejdstid_måned[1] * 0.75

In [ ]:
arbejdstid_måned

In [87]:
# 1.11 - Beregn de samlede omkostninger pr. måned
saml_måned = sum((arbejdstid_måned @ timepris).diagonal())
saml_måned

145112.5078125

In [66]:
# 1.12 - Justér den ugentlige arbejdstid, så James har 25 % færre timer og Sarah har 35 % flere timer.
arbejdstid[0] = arbejdstid[0] * 1.35
arbejdstid[1] = arbejdstid[1] * 0.75

In [67]:
# 1.13 - Beregn omkostningerne til Sarah pr. uge
sarah_uge = timepris[:, 0] @ arbejdstid[0]
sarah_uge

20587.5

In [68]:
# 1.14 - Beregn omkostningerne til James pr. ugen 
james_uge = timepris[:, 1] @ arbejdstid[1]
james_uge

12900.0

# 2 - Lineære Ligningssystemer

I del 2 er fokus på løsning af lineære ligningssystemer.
Vi kommer til at arbejde med række echelon form, løsning via invers matrix, samt bruge sympy til, at tjekke vores løsninger.

I første del er i givet flg. lineære ligningssystem:

$$2x + y - 2z = 3 \\ x - y - z = 0 \\ x + y + 3z = 12 $$

Ligningssystemet kan også skrives som en matrix $A$ og to vektorer $\overrightarrow{x}. \overrightarrow{b}$:
$$\begin{bmatrix} 2 & 1 & -2 \\ 1 & -1 & -1 \\ 1 & 1 & 3 \end{bmatrix} \begin{bmatrix} x \\ y \\ z \end{bmatrix} = \begin{bmatrix} 3 \\ 0 \\ 12 \end{bmatrix}$$.

Eller som en augmented matrix:
$$
\left[\begin{array}{rrr|r}
2 & 1 & -2 & 3 \\ 
1 & -1 & -1 & 0 \\
1 & 1 & 3 & 12
\end{array}\right]
$$

In [7]:
# 2.1 - Definer ligningssystemet som matricen A og vektoren b
A = np.array([[2, 1 , -2],
             [1, -1, -1],
             [1, 1, 3]])

b = np.array([3, 0, 12])

In [8]:
# 2.2 - Beregn den inverse matrix af A og gem den som en variable
inv_A = np.linalg.inv(A)
print(inv_A)

[[ 0.16666667  0.41666667  0.25      ]
 [ 0.33333333 -0.66666667 -0.        ]
 [-0.16666667  0.08333333  0.25      ]]


In [14]:
# 2.3 - Find løsningen til det lineære ligningssystem
linalg.solve(A, b)

array([3.5, 1. , 2.5])

I denne del er i givet flg. lineære ligningssystem:
$$x_1 - 2x_2 + x_3 = 2 \\ 2x_1 - 4x_2 + 3x_3 = 7 \\ -x_1 + 2x_2 =1$$

Ligningssystemet kan også skrives som en matrix og to vektorer:
$$\begin{bmatrix} 1 & -2 & 1 \\ 2 & -4 & 3 \\ -1 & 2 & 0 \end{bmatrix} \begin{bmatrix} x_1 \\ x_2 \\ x_3 \end{bmatrix} = \begin{bmatrix} 2 \\ 7 \\ 1 \end{bmatrix} $$.

Eller som en augmented matrix:
$$
\left[\begin{array}{rrr|r}
1 & -2 & 1 & 2 \\ 
2 & -4 & 3 & 7 \\
-1 & 2 & 0 & 1
\end{array}\right]
$$

In [15]:
# 2.5 - Definer ligningssystemet som en augmented matrix
A2 = np.array([
    [1, -2, 1, 2],
    [2, -4, 3, 7],
    [-1, 2, 0, 1]
])

In [16]:
# 2.6 - Løs ligningssystemet ved at omskrive til række echelon form. 
A2[1] = A2[1] - 2 * A2[0]

In [18]:
A2

array([[ 1, -2,  1,  2],
       [ 0,  0,  1,  3],
       [-1,  2,  0,  1]])

In [20]:
A2[2] = A2[2] + A2[0]
A2

array([[ 1, -2,  1,  2],
       [ 0,  0,  1,  3],
       [ 1, -2,  2,  5]])

*2.7 - Opskriv og test mindst én vallid løsning*
Da de to 

In [22]:
# 2.8 - Definer ligningssystemet som matricen A og vektoren b
A = np.array([
    [1, -2, 3],
    [2, -4, 3],
    [-1, 2, 0]
])


b = np.array([2, 7, 1])

In [23]:
# 2.9 - Beregn løsningen til ligningssystemet vha. A^-1 (den inverse matrice af A)
inv_A = np.linalg.inv(A)
print(inv_A)

LinAlgError: Singular matrix

*2.10 - Hvilket resultat får vi vha. denne løsningsmetode?
        Hvorfor får vi dette resultat?*

Vi kan ikke beregne den inverse matrice $A^{-1}$ da det er en singular matrix. Det vil sige, at mindst én af de tre ligninger kan beskrives som en lineær transformation af de andre.
Derfor kan vi ikke finde en enkelt løsning, men har derimod uendeligt mange løsninger.
Disse løsninger kan så beskrives ud fra nedenstående ligning. 

In [21]:
# 2.9 - Check løsningen vha. sympy

from sympy import * #importer alt fra Sympy
from sympy.solvers.solveset import linsolve #importer linalg solver

x, y, z = symbols('x, y, z') #definer vores symboler

linsolve(Matrix(A2), (x, y, z)) #definer matrix og hvad der skal løses

{(2*y - 1, y, 3)}

*2.10 - Hvad fortæller løsningen os om ligningssystemet?*